
<H3 align='center'> An Attention-Based Architecture for
Hierarchical Classification with CNNs </H3>

<H5 align='center'> FASHION MNIST </H3>

<hr style="height:2px;border:none"/>


# Dependencies

In [ ]:
import keras
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Concatenate, Add, Softmax
from keras.layers import Conv2D, MaxPooling2D, Input, BatchNormalization
from keras.initializers import he_normal
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard, CSVLogger
from keras.utils.data_utils import get_file
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from keras.utils.vis_utils import plot_model
import pickle
from keras.models import load_model
from keras.callbacks import CSVLogger
from tensorflow.keras.utils import set_random_seed
from scipy import stats

# Computes hierarchical metrics proposed by Kiritchenko et al (2005)
def hierarchical_metrics(true,pred):
  true_labels = []
  true_fine = true[2].argmax(axis=1)
  true_c2 = true[1].argmax(axis=1)
  true_c1 = true[0].argmax(axis=1)
  for i in range(len(true_fine)):
    true_labels.append([true_c1[i],true_c2[i],true_fine[i]])
  pred_labels = []
  pred_c1 = pred[0].argmax(axis = 1)
  pred_c2 = pred[1].argmax(axis = 1)
  pred_fine = pred[2].argmax(axis = 1) 
  for i in range(len(pred_c1)):
    pred_labels.append([pred_c1[i],pred_c2[i],pred_fine[i]])
  preci = precision(true_labels,pred_labels)
  reca = recall(true_labels,pred_labels)
  f_1 = f1(true_labels,pred_labels)
  return preci,reca,f_1

# Hierarchical metrics, proposed by Kiritchenko et al (2005)
# Implementation 
# https://gitlab.com/dacs-hpi/hiclass/-/blob/main/hiclass/metrics.py


def precision(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute precision score for hierarchical classification.

    hP = sum(|S intersection T|) / sum(|S|),
    where S is the set consisting of the most specific class(es) predicted for a test example and all respective ancestors
    and T is the set consisting of the true most specific class(es) for a test example and all respective ancestors.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    precision : float
        What proportion of positive identifications was actually correct?
    """
    assert len(y_true) == len(y_pred)
    sum_intersection = 0
    sum_prediction_and_ancestors = 0
    for ground_truth, prediction in zip(y_true, y_pred):
        sum_intersection = sum_intersection + len(
            set(ground_truth).intersection(set(prediction))
        )
        sum_prediction_and_ancestors = sum_prediction_and_ancestors + len(
            set(prediction)
        )
    precision = sum_intersection / sum_prediction_and_ancestors
    return precision


def recall(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute recall score for hierarchical classification.

    hR = sum(|S intersection T|) / sum(|T|),
    where S is the set consisting of the most specific class(es) predicted for a test example and all respective ancestors
    and T is the set consisting of the true most specific class(es) for a test example and all respective ancestors.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    recall : float
        What proportion of actual positives was identified correctly?
    """
    assert len(y_true) == len(y_pred)
    sum_intersection = 0
    sum_prediction_and_ancestors = 0
    for ground_truth, prediction in zip(y_true, y_pred):
        sum_intersection = sum_intersection + len(
            set(ground_truth).intersection(set(prediction))
        )
        sum_prediction_and_ancestors = sum_prediction_and_ancestors + len(
            set(ground_truth)
        )
    recall = sum_intersection / sum_prediction_and_ancestors
    return recall


def f1(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Compute f1 score for hierarchical classification.

    hF = 2 * hP * hR / (hP + hR),
    where hP is the hierarchical precision and hR is the hierarchical recall.

    Parameters
    ----------
    y_true : np.array of shape (n_samples, n_levels)
        Ground truth (correct) labels.
    y_pred : np.array of shape (n_samples, n_levels)
        Predicted labels, as returned by a classifier.
    Returns
    -------
    f1 : float
        Weighted average of the precision and recall
    """
    assert len(y_true) == len(y_pred)
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    f1 = 2 * prec * rec / (prec + rec)
    return f1

# General Settings

In [ ]:
height, width = 28, 28
channel = 1
if K.image_data_format() == 'channels_first':
    input_shape = (channel, height, width)
else:
    input_shape = (height, width, channel)

In [ ]:
train_size = 60000
test_size = 10000

coarse1_classes = 2

coarse2_classes = 6

num_classes  = 10

batch_size   = 128
epochs       = 60

In [ ]:
WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                         WEIGHTS_PATH,
                         cache_subdir='models')
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
class_names_c1 = ['Clothes', 'Goods']
class_names_c2 = ['Tops', 'Bottoms', 'Dresses', 'Outers', 'Accessories', 'Shoes']

In [ ]:
c2_to_c1 = {0:0, 1:0, 2:0, 3:0, 4:1, 5:1}
fine_to_c2 = {0:0, 1:1, 2:0, 3:2, 4:3, 5:5, 6:0, 7:5, 8:4, 9:5}

def print_mappings(mapping, source, dest):
    for k,v in mapping.items():
        print(source[k], "->", dest[v])

print_mappings(c2_to_c1, class_names_c2, class_names_c1)
print("-"*10)
print_mappings(fine_to_c2, class_names, class_names_c2)

train_labels_fine = to_categorical(train_labels)
test_labels_fine = to_categorical(test_labels)
train_labels_c2_index = [fine_to_c2[i] for i in train_labels]
train_labels_c2 = to_categorical(train_labels_c2_index)
test_labels_c2_index = [fine_to_c2[i] for i in test_labels]
test_labels_c2 = to_categorical(test_labels_c2_index)
train_labels_c1_index = [c2_to_c1[i] for i in train_labels_c2_index]
train_labels_c1 = to_categorical(train_labels_c1_index)
test_labels_c1_index = [c2_to_c1[i] for i in test_labels_c2_index]
test_labels_c1 = to_categorical(test_labels_c1_index)



In [ ]:
x_train = train_images[..., np.newaxis]
x_test = test_images[..., np.newaxis]

y_train = train_labels_fine
y_test = test_labels_fine

y_c1_train = train_labels_c1
y_c1_test = test_labels_c1

y_c2_train = train_labels_c2
y_c2_test = test_labels_c2

print("x_train shape: ", x_train.shape)
print("x_test shape: ", x_test.shape)

print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)
print("y_c1_train shape: ", y_c1_train.shape)
print("y_c1_test shape: ", y_c1_test.shape)
print("y_c2_train shape: ", y_c2_train.shape)
print("y_c2_test shape: ", y_c2_test.shape)

In [ ]:
def scheduler(epoch):
  learning_rate_init = 0.001
  if epoch > 42:
    learning_rate_init = 0.0002
  if epoch > 52:
    learning_rate_init = 0.00005
  return learning_rate_init

In [ ]:
class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta, gamma):
    self.alpha = alpha
    self.beta = beta
    self.gamma = gamma
    # customize your behavior
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 15:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.8)
      K.set_value(self.gamma, 0.1)
    if epoch == 25:
      K.set_value(self.alpha, 0.1)
      K.set_value(self.beta, 0.2)
      K.set_value(self.gamma, 0.7)
    if epoch == 35:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 0)
      K.set_value(self.gamma, 1)

# B-CNN Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper 

img_input = Input(shape=input_shape, name='input')
#block 1 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#block 2 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 1 
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

#block 3 
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 2
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(1024, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(1024, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(coarse2_classes, activation='softmax')(c_2_bch)

#block 4 
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)


#block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

#fine 
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax')(x)


model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='hcnn_seo')


#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma], 
              metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_b_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_b_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_b_cnn_c,reca_b_cnn_c,f1_b_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- B-CNN Base C  ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_b_cnn_c[4])
print("Accuracy level 2:",score_b_cnn_c[5])
print("Accuracy level 3:",score_b_cnn_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_b_cnn_c)
print("Recall:",reca_b_cnn_c)
print("f1:",f1_b_cnn_c)
print("Parameters:","{:,}".format(parameters_b_cnn_c))

# BA-CNN Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

# neurons of all dense layers on each branch 
branch_neurons = 256

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper 

img_input = Input(shape=input_shape, name='input')
#block 1 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#block 2 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 1 
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(branch_neurons, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(branch_neurons, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
#c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

#block 3 
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 2
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(branch_neurons, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(branch_neurons, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)


#block 4 
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)


#block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

#fine 
x = Flatten(name='flatten')(x)
x = Dense(branch_neurons, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(branch_neurons, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

#-- Att for coarse 1---
# Coarse 1
sfcn_1_1 = Dense(64, name='fc1_1')(c_1_bch)
sfcn_1_1 = Dense(1, name='fc1_2')(sfcn_1_1)
# Coarse 2
sfcn_1_2 = Dense(64, name='fc1_3')(c_2_bch)
sfcn_1_2 = Dense(1, name='fc1_4')(sfcn_1_2)
# Fine
sfcn_1_3 = Dense(64, name='fc1_5')(x)
sfcn_1_3 = Dense(1, name='fc1_6')(sfcn_1_3)

score_vector_1 = Concatenate()([sfcn_1_1,sfcn_1_2,sfcn_1_3]) # Score vector 1
att_weights_1 = Activation('softmax', name='attention_weights_1')(score_vector_1) # Attention weights 1
weightned_sum_1 = Add()([c_1_bch*att_weights_1[0][0],c_2_bch*att_weights_1[0][1],x*att_weights_1[0][2]]) # Weightned sum 1

# Concat and prediction
coarse_1_concat = Concatenate()([c_1_bch,weightned_sum_1])
c_1_pred = Dense(coarse1_classes, activation='softmax', name='c1_predictions_cifar100')(coarse_1_concat)


#-- Att for coarse 2---

# Coarse 1
sfcn_2_1 = Dense(64, name='fc2_1')(c_1_bch)
sfcn_2_1 = Dense(1, name='fc2_2')(sfcn_2_1)
# Coarse 2
sfcn_2_2 = Dense(64, name='fc2_3')(c_2_bch)
sfcn_2_2 = Dense(1, name='fc2_4')(sfcn_2_2)
# Fine
sfcn_2_3 = Dense(64, name='fc2_5')(x)
sfcn_2_3 = Dense(1, name='fc2_6')(sfcn_2_3)

score_vector_2 = Concatenate()([sfcn_2_1,sfcn_2_2,sfcn_2_3]) # Score vector 1
att_weights_2 = Activation('softmax', name='attention_weights_2')(score_vector_2) # Attention weights 1
weightned_sum_2 = Add()([c_1_bch*att_weights_2[0][0],c_2_bch*att_weights_2[0][1],x*att_weights_2[0][2]]) # Weightned sum 1

# Concat and prediction
coarse_2_concat = Concatenate()([c_2_bch,weightned_sum_2])
c_2_pred = Dense(coarse2_classes, activation='softmax', name='c2_predictions_cifar100')(coarse_2_concat)


#-- Att for fine---

# Coarse 1
sfcn_3_1 = Dense(64, name='fc3_1')(c_1_bch)
sfcn_3_1 = Dense(1, name='fc3_2')(sfcn_3_1)
# Coarse 2
sfcn_3_2 = Dense(64, name='fc3_3')(c_2_bch)
sfcn_3_2 = Dense(1, name='fc3_4')(sfcn_3_2)
# Fine
sfcn_3_3 = Dense(64, name='fc3_5')(x)
sfcn_3_3 = Dense(1, name='fc3_6')(sfcn_3_3)

score_vector_3 = Concatenate()([sfcn_3_1,sfcn_3_2,sfcn_3_3]) # Score vector 1
att_weights_3 = Activation('softmax', name='attention_weights_3')(score_vector_3) # Attention weights 1
weightned_sum_3 = Add()([c_1_bch*att_weights_3[0][0],c_2_bch*att_weights_3[0][1],x*att_weights_3[0][2]]) # Weightned sum 3

# Concat and prediction
fine_concat = Concatenate()([x,weightned_sum_3])
fine_pred = Dense(num_classes, activation='softmax', name='predictions_cifar100')(fine_concat)


model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='bacnn')


#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma], 
              metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_ba_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_ba_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_ba_cnn_c,reca_ba_cnn_c,f1_ba_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- BA-CNN Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_ba_cnn_c[4])
print("Accuracy level 2:",score_ba_cnn_c[5])
print("Accuracy level 3:",score_ba_cnn_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_ba_cnn_c)
print("Recall:",reca_ba_cnn_c)
print("f1:",f1_ba_cnn_c)
print("Parameters:","{:,}".format(parameters_ba_cnn_c))

# H-CNN Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper 

img_input = Input(shape=input_shape, name='input')
#block 1 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#block 2 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 1 
c_1_bch_flatt = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu', name='c1_fc_cifar10_1')(c_1_bch_flatt)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch = Dense(256, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

#block 3 
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 2
c_2_bch_flatt = Flatten(name='c2_flatten')(x)
c_2_bch_concat = Concatenate()([c_1_bch_flatt,c_2_bch_flatt]) # Conectivity Pattern
c_2_bch = Dense(1024, activation='relu', name='c2_fc_cifar100_1')(c_2_bch_concat)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(1024, activation='relu')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(coarse2_classes, activation='softmax')(c_2_bch)

#block 4 
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)


#block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

#fine 
x_flatt = Flatten(name='flatten')(x)
x = Concatenate()([c_2_bch_concat,x_flatt]) # Conectivity Pattern
x = Dense(4096, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax')(x)


model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='hcnn_zhang')


#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma], 
              metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_h_cnn_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_h_cnn_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_h_cnn_c,reca_h_cnn_c,f1_h_cnn_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- H-CNN Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_h_cnn_c[4])
print("Accuracy level 2:",score_h_cnn_c[5])
print("Accuracy level 3:",score_h_cnn_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_h_cnn_c)
print("Recall:",reca_h_cnn_c)
print("f1:",f1_h_cnn_c)
print("Parameters:","{:,}".format(parameters_h_cnn_c))

# Add-net Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  

img_input = Input(shape=input_shape, name='input')
#block 1 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#block 2 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 1 
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(256, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch_out = Dense(256, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch_out)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

#block 3 
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 2
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(256, activation='relu', name='c2_fc_cifar100_1')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(256, activation='relu')(c_2_bch)
c_2_bch_out = Add()([c_2_bch,c_1_bch_out])
c_2_bch = BatchNormalization()(c_2_bch_out)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(coarse2_classes, activation='softmax')(c_2_bch)

#block 4 
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)


#block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

#fine 
x = Flatten(name='flatten')(x)
x = Dense(256, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Add()([x,c_2_bch_out])
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax')(x)


model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='addnet_fashion')


#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma], 
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_addnet_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_addnet_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_addnet_c,reca_addnet_c,f1_addnet_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- Add-net Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_addnet_c[4])
print("Accuracy level 2:",score_addnet_c[5])
print("Accuracy level 3:",score_addnet_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_addnet_c)
print("Recall:",reca_addnet_c)
print("f1:",f1_addnet_c)
print("Parameters:","{:,}".format(parameters_addnet_c))

# Concat-net Base C

In [ ]:
# if True, the model uses BT-strategy for training
bt_strategy = True

In [ ]:
#----------------------- model definition ---------------------------
if bt_strategy == True:
  alpha = K.variable(value=0.98, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.01, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.01, dtype="float32", name="gamma") # A3 in paper
else:
  alpha = K.variable(value=0.33, dtype="float32", name="alpha") # A1 in paper
  beta = K.variable(value=0.33, dtype="float32", name="beta") # A2 in paper
  gamma = K.variable(value=0.34, dtype="float32", name="gamma") # A3 in paper  

img_input = Input(shape=input_shape, name='input')
#block 1 
x = Conv2D(64, (3, 3), activation='relu', padding='same')(img_input)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#block 2 
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 1 
c_1_bch = Flatten(name='c1_flatten')(x)
c_1_bch = Dense(512, activation='relu', name='c1_fc_cifar10_1')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_bch_out = Dense(512, activation='relu')(c_1_bch)
c_1_bch = BatchNormalization()(c_1_bch_out)
c_1_bch = Dropout(0.5)(c_1_bch)
c_1_pred = Dense(coarse1_classes, activation='softmax')(c_1_bch)

#block 3 
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)

#coarse 2
c_2_bch = Flatten(name='c2_flatten')(x)
c_2_bch = Dense(1024, activation='relu', name='c2_fc_cifar100_1')(c_2_bch)
c_2_bch = BatchNormalization()(c_2_bch)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_bch = Dense(1024, activation='relu')(c_2_bch)
c_2_bch_out = Concatenate()([c_2_bch,c_1_bch_out])
c_2_bch = BatchNormalization()(c_2_bch_out)
c_2_bch = Dropout(0.5)(c_2_bch)
c_2_pred = Dense(coarse2_classes, activation='softmax')(c_2_bch)

#block 4 
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2, 2))(x)


#block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)

#fine 
x = Flatten(name='flatten')(x)
x = Dense(4096, activation='relu', name='fc_cifar10_1')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation='relu')(x)
x = Concatenate()([x,c_2_bch_out])
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
fine_pred = Dense(num_classes, activation='softmax')(x)


model = Model(img_input, [c_1_pred, c_2_pred, fine_pred], name='concatnet_fashion')


#----------------------- compile and fit ---------------------------
sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd, 
              loss_weights=[alpha, beta, gamma], 
              # optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

In [ ]:
# Callbacks 
change_lr = LearningRateScheduler(scheduler)
change_lw = LossWeightsModifier(alpha, beta, gamma)

if bt_strategy == True:
  cbks = [change_lr, change_lw]
else:
  cbks = [change_lr]

history_bcnn_b = model.fit(x_train, [y_c1_train, y_c2_train, y_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          callbacks=cbks,
          validation_data=(x_test, [y_c1_test, y_c2_test, y_test]))

score_concatnet_c = model.evaluate(x_test, [y_c1_test, y_c2_test, y_test], verbose=0)
parameters_concatnet_c = np.sum([K.count_params(w) for w in model.trainable_weights])

predictions = model.predict(x_test)
preci_concatnet_c,reca_concatnet_c,f1_concatnet_c= hierarchical_metrics([y_c1_test, y_c2_test, y_test],predictions)

# Results
print("--- Concat-net Base C ---")
print("--- Accuracy per level ---")
print("Accuracy level 1:",score_concatnet_c[4])
print("Accuracy level 2:",score_concatnet_c[5])
print("Accuracy level 3:",score_concatnet_c[6])
print("--- Hierarchical Metrics ---")
print("Precision:",preci_concatnet_c)
print("Recall:",reca_concatnet_c)
print("f1:",f1_concatnet_c)
print("Parameters:","{:,}".format(parameters_concatnet_c))

# Summary

In [ ]:
summary = {'':['B-CNN Base C','BA-CNN Base C ','H-CNN Base C','Add-net Base C','Concat-net Base C'],'Coarse 1': [score_b_cnn_c[4],score_ba_cnn_c[4],score_h_cnn_c[4],score_addnet_c[4],score_concatnet_c[4]],'Coarse 2': [score_b_cnn_c[5],score_ba_cnn_c[5],score_h_cnn_c[5],score_addnet_c[5],score_concatnet_c[5]],'Fine': [score_b_cnn_c[6],score_ba_cnn_c[6],score_h_cnn_c[6],score_addnet_c[6],score_concatnet_c[6]],'Precision':[preci_b_cnn_c,preci_ba_cnn_c,preci_h_cnn_c,preci_addnet_c,preci_concatnet_c],'Recall':[reca_b_cnn_c,reca_ba_cnn_c,reca_h_cnn_c,reca_addnet_c,reca_concatnet_c],'f1':[f1_b_cnn_c,f1_ba_cnn_c,f1_h_cnn_c,f1_addnet_c,f1_concatnet_c],'Parameters': [parameters_b_cnn_c,parameters_ba_cnn_c,parameters_h_cnn_c,parameters_addnet_c,parameters_concatnet_c]}
summary = pd.DataFrame(summary)
summary['Parameters'] = (summary['Parameters'].astype(float)/1000000).round(2).astype(str) + 'MM'
summary = summary.set_index('')
summary.style.highlight_max()